# Calculate synthetic arrival times using a Python script

`Cake` allows to calculate the arrival times of differen seismic phases ([example here](https://pyrocko.org/docs/current/apps/cake/manual.html#calculate-p-phase-arrivals)). In this task we will compute P and S wave arrivals for the station `GE.KTHA` and the 2020 M<sub>w</sub> 7.0 Samos earthquake and compare them to the picks, we made in the previous module.

We start by importing the different modules from Pyrocko, we need:

In [84]:
from pyrocko import cake, model, util
from pyrocko.gui.util import Marker

import numpy as num

In the next step both the event and station data need to be loaded. We make use of `load_one_event` and `load_stations` for this purpose.

In [7]:
# Loading of events
event = model.load_one_event(filename='data/events.yaml')

# Loading of stations
stations = model.load_stations(filename='data/stations.yaml')

In the next step we need to select the station `GE.KTHA`:

In [4]:
station = None

# Iterate through stations and check for matching network-station-location code
for sta in stations:
    if sta.nsl() == ('GE', 'KTHA', ''):
        station = sta
        
        break

and calculate the distance to the earthquake:

In [11]:
distance = event.distance_to(station) * cake.m2d  # Convert meter to degrees for usage in cake

print('Distance between event and station: {:.4f} deg'.format(distance))

Distance between event and station: 3.4046 deg


Great, finally we can calculate the arrival times for the different phases based on **cake**. First the synthetic earth model is loaded:

In [17]:
# Load builtin 'prem-no-ocean' model (medium resolution)
earth_model = cake.load_model('prem-no-ocean.m')

We also need to define the phases, we want to extract the travel time for:

In [13]:
# Define the phase to use.
phases = [
    cake.PhaseDef('P'),
    cake.PhaseDef('S')]

Now arrivals are calulated and we directly feed it into `Snuffler` markers for usage in `Snuffler`. 

In [90]:
markers = []

# calculate distances and arrivals and store them as marker:
print('{:<12}|{:>11}'.format('phase name', 'arrival'))
print('-' * 24)

for arrival in earth_model.arrivals([distance], phases=phases, zstart=event.depth):
    markers.append(Marker(
        tmin=arrival.t + event.time,
        tmax=arrival.t + event.time,
        nslc_ids=[station.nsl() + ('???',)]))
    
    phase = arrival.used_phase()
    print('{:<12}|{:>11}'.format(phase.used_repr(), util.time_to_str(arrival.t + event.time, format='%H:%M:%S')))

phase name  |    arrival
------------------------
P_24.4p     |   11:52:16
P           |   11:52:23
Pv24.4p     |   11:52:23
P           |   11:52:31
Pv15p       |   11:52:31
S_24.4s     |   11:52:56
S           |   11:53:06
Sv24.4s     |   11:53:06
S           |   11:53:24
Sv15s       |   11:53:24
Sv2891s     |   12:06:59


In the last step we repeat the loading of waveforms into `Snuffler` (check module 3 - Introduction to seismological toolboxes).

In [83]:
# Import
from pyrocko.squirrel import Squirrel
from pyrocko.trace import snuffle

# Loading of waveforms
sq = Squirrel()
sq.add('data/')

# Snuffle traces and markers
snuffle(traces=sq.get_waveforms(), markers=markers)

D◷ Gathering file names    working           MD◷ Preparing database      working           MD◷ Indexing files          working           MD◷ Aggregating selection   working           Mcc.py:pyrocko.gui.snuffling.okada - WARNING  - 
 /home/malde/.snufflings/okada/libokada.so: undefined symbol: okada_get_consts
--> run 'make' in okada snuffling directory <--


Finally, compare your picks with the one computed by **cake**. Are they in good agreement? Why could there be differences?